In [5]:
# Binance UM Futures — robust hourly fetch (handles '-1130 startTime invalid')

import pandas as pd
import requests, time, math
from datetime import datetime, timedelta, timezone
from pathlib import Path

SYMBOL      = "BTCUSDT"
START_DATE  = datetime(2020, 1, 1, tzinfo=timezone.utc)   # ask early; code will skip forward if invalid
END_DATE    = datetime.now(timezone.utc)

OUT_ROOT    = Path("../data/")
RAW_DIR     = OUT_ROOT / "derivatives_raw"
FEAT_DIR    = OUT_ROOT / "derivatives_features"
RAW_DIR.mkdir(parents=True, exist_ok=True)
FEAT_DIR.mkdir(parents=True, exist_ok=True)

FAPI = "https://fapi.binance.com"                 # funding
DATA = "https://fapi.binance.com/futures/data"    # OI & ratios
REQ_TIMEOUT = 30
SLEEP = 0.25

def _to_ms(dt): return int(dt.timestamp() * 1000)

def _is_binance_1130(resp):
    try:
        j = resp.json()
    except Exception:
        return False
    return isinstance(j, dict) and j.get("code") == -1130

def _window_iter(start_dt, end_dt, days):
    cur = start_dt
    delta = timedelta(days=days)
    while cur <= end_dt:
        win_end = min(cur + delta, end_dt)
        yield cur, win_end
        # advance 1 ms past win_end to avoid overlap
        cur = (win_end + timedelta(milliseconds=1))

# -------- Funding (8h native) uses start+end and is happy with long ranges --------
def get_funding(symbol=SYMBOL, start=START_DATE, end=END_DATE, sleep=SLEEP):
    url = f"{FAPI}/fapi/v1/fundingRate"
    out = []
    cursor = start
    while cursor <= end:
        # 90d window is safe for this endpoint
        win_end = min(cursor + timedelta(days=90), end)
        params = {
            "symbol": symbol,
            "startTime": _to_ms(cursor),
            "endTime": _to_ms(win_end),
            "limit": 1000
        }
        r = requests.get(url, params=params, timeout=REQ_TIMEOUT)
        if _is_binance_1130(r):
            # Move forward (symbol might not have data that early)
            cursor = cursor + timedelta(days=90)
            time.sleep(sleep)
            continue
        r.raise_for_status()
        batch = r.json()
        if not batch:
            # no more data in this window; jump to next window
            cursor = win_end + timedelta(milliseconds=1)
            time.sleep(sleep)
            continue
        out.extend(batch)
        last_ms = batch[-1]["fundingTime"]
        cursor = datetime.fromtimestamp(last_ms/1000, tz=timezone.utc) + timedelta(milliseconds=1)
        time.sleep(sleep)

    df = pd.DataFrame(out)
    if df.empty: return df
    df["fundingTime"] = pd.to_datetime(df["fundingTime"], unit="ms", utc=True)
    for c in ["fundingRate","markPrice"]:
        if c in df: df[c] = pd.to_numeric(df[c], errors="coerce")
    keep = [c for c in ["symbol","fundingTime","fundingRate","markPrice"] if c in df]
    return df[keep].sort_values("fundingTime").reset_index(drop=True)

# -------- Helper for endpoints that REQUIRE startTime & endTime and reject too-early times --------
def _windowed_get_with_skip(url, base_params, time_key, start_dt, end_dt, window_days=30, limit=500, sleep=SLEEP):
    out = []
    for ws, we in _window_iter(start_dt, end_dt, window_days):
        params = dict(base_params)
        params.update({"startTime": _to_ms(ws), "endTime": _to_ms(we), "limit": limit})
        r = requests.get(url, params=params, timeout=REQ_TIMEOUT)
        if _is_binance_1130(r):
            # This whole window starts before supported history; skip it forward
            time.sleep(sleep)
            continue
        r.raise_for_status()
        data = r.json()
        batch = data if isinstance(data, list) else data.get("data", []) or []
        if not batch:
            # Nothing here; continue to next window
            time.sleep(sleep)
            continue
        out.extend(batch)
        time.sleep(sleep)
    # de-dup in case of any boundary overlaps
    return out

# -------- Open Interest (1h) --------
def get_open_interest(symbol=SYMBOL, start=START_DATE, end=END_DATE, period="1h"):
    """
    /futures/data/openInterestHist
    Accepts symbol, period, startTime, endTime, limit (<=500).
    We walk in 30d windows and skip invalid early ranges automatically.
    """
    url = f"{DATA}/openInterestHist"
    rows = _windowed_get_with_skip(
        url, {"symbol": symbol, "period": period},
        time_key="timestamp",
        start_dt=start, end_dt=end,
        window_days=30, limit=500, sleep=SLEEP
    )
    df = pd.DataFrame(rows)
    if df.empty: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    for c in ["sumOpenInterest","sumOpenInterestValue"]:
        if c in df: df[c] = pd.to_numeric(df[c], errors="coerce")
    keep = [c for c in ["symbol","timestamp","sumOpenInterest","sumOpenInterestValue"] if c in df]
    return df[keep].drop_duplicates(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)

# -------- Taker long/short ratio (1h) --------
def get_taker_ratio(symbol=SYMBOL, start=START_DATE, end=END_DATE, period="1h"):
    """
    /futures/data/topLongShortAccountRatio
    Same behavior as OI. Use 30d windows + auto-skip when -1130 occurs.
    """
    url = f"{DATA}/topLongShortAccountRatio"
    rows = _windowed_get_with_skip(
        url, {"symbol": symbol, "period": period},
        time_key="timestamp",
        start_dt=start, end_dt=end,
        window_days=30, limit=500, sleep=SLEEP
    )
    df = pd.DataFrame(rows)
    if df.empty: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    for c in ["longShortRatio","longAccount","shortAccount"]:
        if c in df: df[c] = pd.to_numeric(df[c], errors="coerce")
    keep = [c for c in ["symbol","timestamp","longShortRatio","longAccount","shortAccount"] if c in df]
    return df[keep].drop_duplicates(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)

# ---------------- Run & Save RAW ----------------
print("Fetching funding (8h native)...")
funding_raw = get_funding()
print("Funding rows:", len(funding_raw))

print("Fetching open interest (1h)...")
oi_raw = get_open_interest(period="1h")
print("Open interest rows:", len(oi_raw))

print("Fetching taker long/short ratio (1h)...")
taker_raw = get_taker_ratio(period="1h")
print("Taker rows:", len(taker_raw))

funding_raw.to_csv(RAW_DIR / f"binance_funding_{SYMBOL}.csv", index=False)
oi_raw.to_csv(RAW_DIR / f"binance_open_interest_{SYMBOL}_1h.csv", index=False)
taker_raw.to_csv(RAW_DIR / f"binance_taker_longshort_{SYMBOL}_1h.csv", index=False)

# ---------------- Build hourly features ----------------
# Funding -> hourly grid via ffill (max 8H)
if not funding_raw.empty:
    funding_h = (
        funding_raw.rename(columns={"fundingTime":"timestamp"})
        .set_index("timestamp").sort_index()
        .asfreq("1H").ffill(limit=8).reset_index()
    )
    if "fundingRate" in funding_h:
        funding_h["funding_annualized_pct"] = funding_h["fundingRate"] * 3 * 365 * 100
else:
    funding_h = pd.DataFrame(columns=["timestamp","symbol","fundingRate","markPrice","funding_annualized_pct"])

# OI hourly + diffs
if not oi_raw.empty:
    oi_h = oi_raw.copy()
    oi_h["oi_change"] = oi_h["sumOpenInterest"].diff()
    oi_h["oi_value_change"] = oi_h["sumOpenInterestValue"].diff()
else:
    oi_h = pd.DataFrame(columns=["timestamp","symbol","sumOpenInterest","sumOpenInterestValue","oi_change","oi_value_change"])

# Taker hourly + shares
if not taker_raw.empty:
    taker_h = taker_raw.copy()
    if "longShortRatio" in taker_h:
        taker_h["long_share"]  = taker_h["longShortRatio"] / (1.0 + taker_h["longShortRatio"])
        taker_h["short_share"] = 1.0 - taker_h["long_share"]
else:
    taker_h = pd.DataFrame(columns=["timestamp","symbol","longShortRatio","longAccount","shortAccount","long_share","short_share"])

# Merge to hourly index
hourly_index = pd.date_range(START_DATE, END_DATE, freq="1H", tz="UTC")
feat = pd.DataFrame(index=hourly_index)

def idx(df): return df.set_index("timestamp").sort_index()

if not funding_h.empty:
    feat = feat.join(idx(funding_h)[["fundingRate","funding_annualized_pct","markPrice"]], how="left")
if not oi_h.empty:
    feat = feat.join(idx(oi_h)[["sumOpenInterest","sumOpenInterestValue","oi_change","oi_value_change"]], how="left")
if not taker_h.empty:
    cols = [c for c in ["longShortRatio","longAccount","shortAccount","long_share","short_share"] if c in taker_h]
    feat = feat.join(idx(taker_h)[cols], how="left")

feat["symbol"] = SYMBOL
feat = feat.reset_index().rename(columns={"index":"timestamp"}).sort_values("timestamp")

# light ffill for tiny gaps
for c in ["fundingRate","funding_annualized_pct","markPrice",
          "sumOpenInterest","sumOpenInterestValue","oi_change","oi_value_change",
          "longShortRatio","longAccount","shortAccount","long_share","short_share"]:
    if c in feat: feat[c] = feat[c].ffill(limit=6)

feat_csv = FEAT_DIR / f"binance_derivatives_features_{SYMBOL}_1h.csv"
feat.to_csv(feat_csv, index=False)
print(f"Saved FEATURES: {feat_csv}  ({len(feat)} rows)")

Fetching funding (8h native)...
Funding rows: 6173
Fetching open interest (1h)...
Open interest rows: 418
Fetching taker long/short ratio (1h)...
Taker rows: 418
Saved FEATURES: ../data/derivatives_features/binance_derivatives_features_BTCUSDT_1h.csv  (49378 rows)


/var/folders/tz/yhx353f17tj_1s1dj9_gw3x80000gn/T/ipykernel_19497/1912326733.py:170: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .asfreq("1H").ffill(limit=8).reset_index()
/var/folders/tz/yhx353f17tj_1s1dj9_gw3x80000gn/T/ipykernel_19497/1912326733.py:195: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_index = pd.date_range(START_DATE, END_DATE, freq="1H", tz="UTC")
